# **Intelligence artificielle pour l’estimation de l’ age basé sur le visage**

Importation des librairies


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential



# **Paramètrage**

In [ ]:
train_path = "dataset/age_prediction_up/age_prediction/train"
test_path = "dataset/age_prediction_up/age_prediction/test"

train_dir = pathlib.Path(train_path)
test_dir = pathlib.Path(test_path)

img_width = 224
img_height = 224
batch_size = 32


epochs = 50


num_classes = 6


# **Fonction permettant de prendre un echantillons d'images**

In [ ]:
def sample_files(directory, sample_fraction):
    all_files = list(directory.glob('*/*.jpg')) 
    sample_size = int(len(all_files) * sample_fraction)
    sampled_files = random.sample(all_files, sample_size)

    return sampled_files


In [ ]:
sampled_train_files = sample_files(train_dir, sample_fraction=0.3) # 70% train
sampled_test_files = sample_files(test_dir, sample_fraction=0.8) # 20% test

print(sampled_train_files)
print(len(sampled_test_files))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)  # Normalisation des valeurs de pixel
test_datagen = ImageDataGenerator(rescale=1./255)   # Normalisation des valeurs de pixel


# **Mettre en format DataFrame pour manipuler des données**

In [ ]:
import pandas as pd

def create_data_frame(files):
    data = {'filename': [], 'class': []}
    for file in files:
        data['filename'].append(str(file))
        data['class'].append(str(file.parent.name))  
    return pd.DataFrame(data)

train_df = create_data_frame(sampled_train_files)
test_df = create_data_frame(sampled_test_files)
class_names = ['0_10', '10_18', '18_30', '30_50', '50_70', '70_100']

In [ ]:

train_gen = train_datagen.flow_from_dataframe(
  train_df,
  target_size=(img_height, img_width),
  batch_size=batch_size,
  class_mode='sparse')

val_gen = test_datagen.flow_from_dataframe(
  test_df,
  target_size=(img_height, img_width),
  batch_size=batch_size,
  class_mode='sparse')

# **Architecture du CNN**


In [ ]:

model = Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes)
])

In [ ]:

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# **Entrainement**

In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=train_gen.samples // batch_size,
    epochs=epochs,
    validation_data=val_gen,
    validation_steps=val_gen.samples // batch_size)


# **Afficher la précision et sauvegarder le model**

In [ ]:
test_loss, test_acc = model.evaluate(val_gen, verbose=2)
print('Test accuracy:', test_acc)

model.save("/model/model_normal_27_07_1.h5")